In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hacker-earth-will-your-employees-leave-you/sample_submission.csv
/kaggle/input/hacker-earth-will-your-employees-leave-you/Test.csv
/kaggle/input/hacker-earth-will-your-employees-leave-you/Train.csv


In [34]:
#importing required packages
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [3]:
#load train, test data
train = pd.read_csv("/kaggle/input/hacker-earth-will-your-employees-leave-you/Train.csv")
test = pd.read_csv("/kaggle/input/hacker-earth-will-your-employees-leave-you/Test.csv")
submit = pd.read_csv("/kaggle/input/hacker-earth-will-your-employees-leave-you/sample_submission.csv")

In [4]:
#shapes
train.shape, test.shape, submit.shape

((7000, 24), (3000, 23), (5, 2))

In [5]:
#preserve test IDs to make submission file at the end
test_IDs = test.Employee_ID

In [6]:
train.head(4)

,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,...,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
0,EID_23371,F,42.0,4,Married,Franklin,IT,Conceptual,4.0,4,...,type2,3.0,4,0.7516,1.8688,2.0,4,5,3,0.1841
1,EID_18000,M,24.0,3,Single,Springfield,Logistics,Analytical,5.0,4,...,type2,4.0,3,-0.9612,-0.4537,2.0,3,5,3,0.0670
2,EID_3891,F,58.0,3,Married,Clinton,Quality,Conceptual,27.0,3,...,type2,1.0,4,-0.9612,-0.4537,3.0,3,8,3,0.0851
3,EID_17492,F,26.0,3,Single,Lebanon,Human Resource Management,Behavioral,4.0,3,...,type2,1.0,3,-1.8176,-0.4537,NaN,3,7,3,0.0668


In [7]:
i = train.pop("Employee_ID")
i = test.pop("Employee_ID")
# i = test.pop("Hometown")
# i = train.pop("Hometown")

### **Merge train and test data**

In [8]:
train["Mark"] = 1
test["Mark"] = 0
data = pd.concat([train, test])
data.shape

(10000, 24)

In [9]:
data.tail()

,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,...,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate,Mark
2995,F,32.0,3,Single,Franklin,Sales,Directive,3.0,3,38,...,2.0,1,-0.1048,0.7075,1.0,4,7,4,NaN,0
2996,F,NaN,2,Single,Franklin,Marketing,Conceptual,10.0,2,57,...,1.0,3,0.7516,0.7075,3.0,4,8,3,NaN,0
2997,F,60.0,4,Single,Lebanon,Logistics,Analytical,35.0,1,69,...,3.0,3,-1.8176,-0.4537,NaN,4,8,4,NaN,0
2998,F,51.0,1,Married,Springfield,IT,Behavioral,23.0,1,30,...,1.0,5,0.7516,-0.4537,2.0,3,9,2,NaN,0
2999,F,45.0,1,Single,Franklin,Sales,Directive,12.0,4,70,...,4.0,3,0.7516,-0.4537,2.0,2,8,5,NaN,0


In [10]:
data = data.reset_index()

In [11]:
#fil Nan values in combined data
data["Age"].fillna(np.ceil(data.Age.mean()), inplace=True)
data["Time_of_service"].fillna(np.ceil(data.Time_of_service.mean()), inplace=True)
data["Work_Life_balance"].fillna(np.ceil(data.Work_Life_balance.mean()), inplace=True)

In [12]:
data.isnull().sum()

index                           0
Gender                          0
Age                             0
Education_Level                 0
Relationship_Status             0
Hometown                        0
Unit                            0
Decision_skill_possess          0
Time_of_service                 0
Time_since_promotion            0
growth_rate                     0
Travel_Rate                     0
Post_Level                      0
Pay_Scale                      12
Compensation_and_Benefits       0
Work_Life_balance               0
VAR1                            0
VAR2                          794
VAR3                            0
VAR4                          954
VAR5                            0
VAR6                            0
VAR7                            0
Attrition_rate               3000
Mark                            0
dtype: int64

In [13]:
data.VAR2.fillna(data.VAR2.mean(), inplace=True)
data.VAR4.fillna(data.VAR4.mean(), inplace=True)
data.Pay_Scale.fillna(np.ceil(data.Work_Life_balance.mean()), inplace=True)

In [14]:
data.shape

(10000, 25)

In [15]:
data.head(3)

,index,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,...,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate,Mark
0,0,F,42.0,4,Married,Franklin,IT,Conceptual,4.0,4,...,3.0,4,0.7516,1.8688,2.0,4,5,3,0.1841,1
1,1,M,24.0,3,Single,Springfield,Logistics,Analytical,5.0,4,...,4.0,3,-0.9612,-0.4537,2.0,3,5,3,0.0670,1
2,2,F,58.0,3,Married,Clinton,Quality,Conceptual,27.0,3,...,1.0,4,-0.9612,-0.4537,3.0,3,8,3,0.0851,1


# Label Encoder

In [16]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()

# data.Gender  = le.fit_transform(data.Gender)
# data.Relationship_Status  = le.fit_transform(data.Relationship_Status)
# data.Hometown  = le.fit_transform(data.Hometown)
# data.Unit  = le.fit_transform(data.Unit)
# data.Decision_skill_possess = le.fit_transform(data.Decision_skill_possess)
# data.Compensation_and_Benefits = le.fit_transform(data.Compensation_and_Benefits)

# One Hot Encoder

In [17]:
dum_df = pd.get_dummies(data.Gender)
data = data.join(dum_df)
dum_df = pd.get_dummies(data.Relationship_Status)
data = data.join(dum_df)
dum_df = pd.get_dummies(data.Hometown)
data = data.join(dum_df)
dum_df = pd.get_dummies(data.Unit)
data = data.join(dum_df)
dum_df = pd.get_dummies(data.Decision_skill_possess)
data = data.join(dum_df)
dum_df = pd.get_dummies(data.Compensation_and_Benefits)
data = data.join(dum_df)

In [18]:
data.head(2)

,index,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,...,Security,Analytical,Behavioral,Conceptual,Directive,type0,type1,type2,type3,type4
0,0,F,42.0,4,Married,Franklin,IT,Conceptual,4.0,4,...,0,0,0,1,0,0,0,1,0,0
1,1,M,24.0,3,Single,Springfield,Logistics,Analytical,5.0,4,...,0,1,0,0,0,0,0,1,0,0


In [19]:
data.shape

(10000, 55)

In [20]:
data = data.drop(['Gender','Relationship_Status','Hometown','Unit','Decision_skill_possess','Compensation_and_Benefits'],axis=1)
# data = data.drop(['Gender','Unit','Compensation_and_Benefits'],axis=1)

In [22]:
# data = data.drop(['F','Single','Washington','Sales','Directive','type4'],axis=1)
# data = data.drop(['F','Sales','type4'],axis=1)

In [23]:
data.dtypes

index                          int64
Age                          float64
Education_Level                int64
Time_of_service              float64
Time_since_promotion           int64
growth_rate                    int64
Travel_Rate                    int64
Post_Level                     int64
Pay_Scale                    float64
Work_Life_balance            float64
VAR1                           int64
VAR2                         float64
VAR3                         float64
VAR4                         float64
VAR5                           int64
VAR6                           int64
VAR7                           int64
Attrition_rate               float64
Mark                           int64
F                              uint8
M                              uint8
Married                        uint8
Single                         uint8
Clinton                        uint8
Franklin                       uint8
Lebanon                        uint8
Springfield                    uint8
W

In [24]:
data = data.reset_index()
data = data.drop(['level_0','index'],axis=1)

In [25]:
data.head()

,Age,Education_Level,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Work_Life_balance,VAR1,...,Security,Analytical,Behavioral,Conceptual,Directive,type0,type1,type2,type3,type4
0,42.0,4,4.0,4,33,1,1,7.0,3.0,4,...,0,0,0,1,0,0,0,1,0,0
1,24.0,3,5.0,4,36,0,3,6.0,4.0,3,...,0,1,0,0,0,0,0,1,0,0
2,58.0,3,27.0,3,51,0,2,8.0,1.0,4,...,0,0,0,1,0,0,0,1,0,0
3,26.0,3,4.0,3,56,1,3,8.0,1.0,3,...,0,0,1,0,0,0,0,1,0,0
4,31.0,1,5.0,4,62,1,3,2.0,3.0,1,...,0,0,0,1,0,0,0,0,1,0


In [30]:
#Separating test and train from data combined 
train = data[data.Mark == 1]
test = data[data.Mark == 0]
train.shape, test.shape

((7000, 48), (3000, 48))

In [31]:
test.columns

Index(['Age', 'Education_Level', 'Time_of_service', 'Time_since_promotion',
       'growth_rate', 'Travel_Rate', 'Post_Level', 'Pay_Scale',
       'Work_Life_balance', 'VAR1', 'VAR2', 'VAR3', 'VAR4', 'VAR5', 'VAR6',
       'VAR7', 'Attrition_rate', 'Mark', 'F', 'M', 'Married', 'Single',
       'Clinton', 'Franklin', 'Lebanon', 'Springfield', 'Washington',
       'Accounting and Finance', 'Human Resource Management', 'IT',
       'Logistics', 'Marketing', 'Operarions', 'Production', 'Purchasing',
       'Quality', 'R&D', 'Sales', 'Security', 'Analytical', 'Behavioral',
       'Conceptual', 'Directive', 'type0', 'type1', 'type2', 'type3', 'type4'],
      dtype='object')

In [32]:
test = test.reset_index()
test = test.drop(['index'],axis=1)
train = train.drop(['Mark'],axis=1)
test = test.drop(['Mark','Attrition_rate'],axis=1)

In [35]:
#preparing for train_test_split
X = train.drop('Attrition_rate',axis=1)
Y = train.Attrition_rate
X.shape, test.shape

((7000, 46), (3000, 46))

In [37]:
# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
# x_train.shape, y_train.shape, x_test.shape

# PCA

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)
X_train.shape, test.shape

((5250, 46), (3000, 46))

In [39]:
from sklearn.preprocessing import StandardScaler

#apply standard scaler on X_train, X_test, test data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
my_test = sc.transform(test)

In [40]:
from sklearn.decomposition import PCA

#applying PCA on X_train and X_test
pca = PCA()
X_train1 = pca.fit_transform(X_train)
X_test1 = pca.transform(X_test)

In [41]:
#checkout the explained variance ratio
explained_variance = pca.explained_variance_ratio_
explained_variance

array([4.40855436e-02, 4.37146632e-02, 4.21644304e-02, 3.99944703e-02,
       3.08663848e-02, 2.96748545e-02, 2.93807419e-02, 2.90049901e-02,
       2.87007192e-02, 2.74014632e-02, 2.66960994e-02, 2.59538434e-02,
       2.54666356e-02, 2.52951625e-02, 2.44233774e-02, 2.42304695e-02,
       2.39476827e-02, 2.38345943e-02, 2.37442685e-02, 2.35226545e-02,
       2.33493968e-02, 2.30985122e-02, 2.26086810e-02, 2.25768485e-02,
       2.24122170e-02, 2.21404831e-02, 2.20223598e-02, 2.19383575e-02,
       2.15896643e-02, 2.14869395e-02, 2.13150413e-02, 2.11600375e-02,
       2.07382265e-02, 2.02592629e-02, 2.00528256e-02, 1.99810947e-02,
       1.97408459e-02, 1.96966829e-02, 1.92742202e-02, 2.45525372e-03,
       1.04816663e-32, 4.06841871e-33, 3.02135889e-33, 1.76995444e-33,
       1.04997002e-33, 4.18922696e-34])

In [42]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_train1 = pca.fit_transform(X_train)
X_test1 = pca.transform(X_test)


In [43]:
X_train1.shape,X_test1.shape

((5250, 2), (1750, 2))

In [44]:
# 22 24 27 30 60

In [45]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
# rgr = GradientBoostingRegressor()

# rgr = XGBRegressor(max_depth=0,subsample=0.1, random_state=0)
# rgr1 = LinearRegression()

# from sklearn.tree import DecisionTreeRegressor
# rgr = DecisionTreeRegressor(random_state=0,max_depth = 1,min_samples_split=30,min_samples_leaf=27)
# rgr = RandomForestRegressor(max_depth=1, n_estimators=1, max_leaf_nodes =100, random_state=0)


from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
rgr = AdaBoostRegressor(random_state=0, n_estimators=11,learning_rate = 0.01)
rgr.fit(X_train1, y_train)
# rgr1.fit(X_train1, y_train)

# Predicting the Test set results
y_pred = rgr.predict(X_test1)
# y_pred1 = rgr1.predict(X_test1)
# metrics.mean_squared_error(y_pred, y_test)

In [48]:
#rmse
metrics.mean_squared_error(y_pred, y_test)

0.035398506017448594

In [53]:
#use when compiling results from two model together
#--------------------------------------------------


# test_modi = sc.transform(test)
# test1 = pca.transform(test_modi)
# res = rgr.predict(test1)
# # res2 = rgr1.predict(test1)
# ans = []
# for i in range(len(res)):
#     ans.append((res[i]+res2[i])/2.0)

## submission file

In [ ]:
subm = pd.DataFrame()
subm["Employee_ID"]=test_IDs
subm["Attrition_rate"] = res
subm.to_csv("result77.csv",index=False)

# Random search

In [55]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.ensemble import GradientBoostingRegressor
# num_estimators = [10,100]
# learn_rates = [0.01, 0.7]
# max_depths = [1, 6]
# min_samples_leaf = [5,10]
# min_samples_split = [5,10]

# param_grid = {'n_estimators': num_estimators,
#               'learning_rate': learn_rates,
#               'max_depth': max_depths,
#               'min_samples_leaf': min_samples_leaf,
#               'min_samples_split': min_samples_split}

# random_search =RandomizedSearchCV(GradientBoostingRegressor(), param_grid, random_state=1, n_iter=100, cv=5, verbose=0, n_jobs=-1)

# random_search.fit(X_train1, y_train)
#random_search.best_params_

In [56]:
# gboost_score=random_search.score(X_train1, y_train)
# gboost_score

In [ ]:
print("Sleeping...")
time.sleep(3*60)
print("Done..!")